<a name='top'></a>    
<div style="width:900px;background:#fdf0db;border:1px solid black;text-align:left;padding:8px;">
    <p>
        <span style="font-size:14pt">
            <b>Scraping from www.mycareersfuture.sg (Search Page Results)</b>
        </span>
    </p>
</div>

<div style="width:900px;background:#f2f2f2;border:1px solid black;text-align:left;padding:8px;">

This notebook file will extract detail information from Search Page Results.
The URL's and The last page of the results are stored in list variables and will be fed to the user defined function webscraper().


In order to run this project please execute the following:
1. Run 01-webscrape-pages.ipynb
2. Run 02-webscrape-details.ipynb
3. Move all output files created by these two notebooks to data folder.
4. Run 03-data-val-and-cleaning.ipynb
   - This notebook will create a final csv file in data folder.
5. Run 04-data-modelling.ipynb
   - This notebook will read the final csv file to perform analysis.
   
Please note that All output files produced by the first two notebooks (01-webscrape-pages.ipynb & 02-webscrape-details.ipynb) will be written to the current directory.  

In [1]:
import os
from selenium import webdriver
# Import sleep:
from time import sleep
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import re

### function webscraper() 
>Used to request search page and extract values

In [20]:
## function to scan website 
def webscraper(url,pages,outfile='outfile.csv'):
    job_titles=[]
    companys=[]    
    locations=[]
    employment_types=[]
    gov_support=[]
    industrys=[]
    salary_units=[]
    salary=[]
    links=[]
    rec_count=0

    #url='https://www.mycareersfuture.sg/search?search=data%20science&sortBy=relevancy&page={}'
    #driver = webdriver.Chrome(executable_path="./chromedriver")

    chromedriver = r"C:\Users\roni_\Documents\1Roni\GA\Install\chromedriver\chromedriver.exe"
    driver = webdriver.Chrome(executable_path=chromedriver)

    for page in range(0,pages+1):
        
        print('Processing Page {}'.format(page),'...')  
        # make request for that page
        driver.get(url.format(page))
        sleep(10)               
        #print(url)

        html=driver.page_source
        # turn into a BeautifulSoup object
        html=BeautifulSoup(html, 'lxml')
        
        for entry in html.find_all('a', {'class':'bg-white mb3 w-100 dib v-top pa3 no-underline flex-ns flex-wrap JobCard__card___22xP3'}):
            # Grab the Jobtitle.
            try:
                job_title = entry.find('h1', {'class': 'f4-5 fw6 mv0 brand-sec dib mr2 JobCard__jobtitle___3HqOw'}).text
                job_titles.append(job_title)
                #print(job_title)
                rec_count=rec_count+1
            except:
                break 
        
            # Grab the Company.
            try:
                company = entry.find('p', {'class': 'f6 fw6 mv0 black-80 mr2 di ttu'}).text
                companys.append(company)
            except:
                companys.append('NA')
                
            #print(company)
        
            #location
            try:
                location=entry.find('p',{'class': 'black-80 f6 fw4 mt0 mb1 dib pr3 icon-bw-location'}).text
                locations.append(location)
            except:
                locations.append('NA')
            #print(location)
        
            #employment_type
            try:
                employment_type=entry.find('p',{'class':'black-80 f6 fw4 mt0 mb1 dib pr3 icon-bw-seniority'}).text
                employment_types.append(employment_type)
            except:
                employment_types.append('NA')
        
            # gov_support
            try:
                #gov_supp=entry.find('p',{'class':'green mv0 pr3 f5-5 fw4 mr2 db pb2 JobCard__greentick___3e2_Y'}).text
                gov_supp=entry.find('p',{'class':'green mv0 pr3 f6 fw4 mr2 db pb2 JobCard__greentick___3e2_Y'}).text
                gov_support.append(gov_supp)
            except:
                gov_supp='NA'   ## roni
                gov_support.append('NA')
            
            #industry:
            try:
                industry=entry.find('p',{'class':'black-80 f6 fw4 mt0 mb1 dib pr3 icon-bw-category'}).text
                industrys.append(industry)
            except:
                industrys.append('NA')
                
            #salary_unit:
            try:
                salary_unit=entry.find('span',{'class':'salary_type black-60 mv0 fw6 f5-5 lh-solid db pb2 dib i'}).text
                salary_units.append(salary_unit)
            except:
                salary_units.append('NA')
                
            #salary:
            for row in entry.findAll("span", class_ = re.compile("salary_range")):
                salary.append(row.text)
            
            #links
            link=entry.get('href')
            link='https://www.mycareersfuture.sg'+str(link)
            links.append(link)
            
            #raj test
            print(gov_supp,job_title)

    #writefile(outfile, job_titles,companys,location,employment_type,gov_support,industrys,salary,links)
    writefile(outfile, job_titles, companys, locations, employment_types, gov_support, industrys, salary, salary_units, links)
    driver.close()
    print('   {} Records written:'.format(outfile),rec_count)
        

### function writefile() 
>Used to write details we have scraped to an output CSV file

In [3]:
## Create an empty DataFrame.
def writefile(filename, 
              job_titles,
              companys,
              locations,
              employment_types,
              gov_support,
              industrys,
              salary,
              salary_units,
              links):
    
    # First, create an empty DataFrame.
    import pandas as pd
    outfile=''
    outfile = pd.DataFrame \
       (columns=["job_titles","companys", "location", 
                 "employment_type", "gov_support", "salary", 
                 "salary_unit", "links"])

    outfile['job_titles']=job_titles
    outfile['companys']=companys
    outfile['location']=locations
    outfile['employment_type']=employment_types
    outfile['gov_support']=gov_support
    outfile['industrys']=industrys
    outfile['salary']=salary
    outfile['salary_unit']=salary_units
    outfile['links']=links

    outfile.to_csv(filename)

In [11]:
## Search strings with not so large number of pages
mylinks=['https://www.mycareersfuture.sg/search?search=data%20science&sortBy=relevancy&page={}',
         'https://www.mycareersfuture.sg/search?search=data%20engineer&sortBy=relevancy&page={}',
         'https://www.mycareersfuture.sg/search?search=data%20scientist&sortBy=relevancy&page={}',
         'https://www.mycareersfuture.sg/search?search=data%20analyst&sortBy=relevancy&page={}',
         'https://www.mycareersfuture.sg/search?search=data%20management&sortBy=relevancy&page={}',
         'https://www.mycareersfuture.sg/search?search=deep%20learning&sortBy=relevancy&page={}']

linkpages=[6, 7, 6, 6, 7, 4]

In [12]:
##Search strings with large number of pages
mylinks2=['https://www.mycareersfuture.sg/search?search=python%09%09%09&sortBy=relevancy&page={}',
         'https://www.mycareersfuture.sg/search?search=big%20data&sortBy=relevancy&page={}',
         'https://www.mycareersfuture.sg/search?search=machine%20learning%20&sortBy=relevancy&page={}',
         'https://www.mycareersfuture.sg/search?search=data%20analyst&sortBy=relevancy&page={}',
         'https://www.mycareersfuture.sg/search?search=data%20consultant&sortBy=relevancy&page={}']

linkpages2=[11, 15, 12, 4, 2]

In [7]:
for i in range(len(mylinks)):
    file='outfile_a0' + str(i) + '.csv'
    print('-------------------------------')
    webscraper(mylinks[i],linkpages[i],file)

-------------------------------
Processing Page 0 ...
Processing Page 1 ...
Processing Page 2 ...
Processing Page 3 ...
Processing Page 4 ...
Processing Page 5 ...
Processing Page 6 ...
   outfile_00.csv Records written: 140
-------------------------------
Processing Page 0 ...
Processing Page 1 ...
Processing Page 2 ...
Processing Page 3 ...
Processing Page 4 ...
Processing Page 5 ...
Processing Page 6 ...
Processing Page 7 ...
   outfile_01.csv Records written: 137
-------------------------------
Processing Page 0 ...
Processing Page 1 ...
Processing Page 2 ...
Processing Page 3 ...
Processing Page 4 ...
Processing Page 5 ...
Processing Page 6 ...
   outfile_02.csv Records written: 116
-------------------------------
Processing Page 0 ...
Processing Page 1 ...
Processing Page 2 ...
Processing Page 3 ...
Processing Page 4 ...
Processing Page 5 ...
Processing Page 6 ...
   outfile_03.csv Records written: 104
-------------------------------
Processing Page 0 ...
Processing Page 1 ...
Pr

In [14]:
for i in range(len(mylinks2)):
    file='outfile_a0' + str(i) + '.csv'
    print('-------------------------------')
    webscraper(mylinks2[i],linkpages2[i],file)

-------------------------------
Processing Page 0 ...
Processing Page 1 ...
Processing Page 2 ...
Processing Page 3 ...
Processing Page 4 ...
Processing Page 5 ...
Processing Page 6 ...
Processing Page 7 ...
Processing Page 8 ...
Processing Page 9 ...
Processing Page 10 ...
Processing Page 11 ...
   outfile_a00.csv Records written: 236
-------------------------------
Processing Page 0 ...
Processing Page 1 ...
Processing Page 2 ...
Processing Page 3 ...
Processing Page 4 ...
Processing Page 5 ...
Processing Page 6 ...
Processing Page 7 ...
Processing Page 8 ...
Processing Page 9 ...
Processing Page 10 ...
Processing Page 11 ...
Processing Page 12 ...
Processing Page 13 ...
Processing Page 14 ...
Processing Page 15 ...
   outfile_a01.csv Records written: 266
-------------------------------
Processing Page 0 ...
Processing Page 1 ...
Processing Page 2 ...
Processing Page 3 ...
Processing Page 4 ...
Processing Page 5 ...
Processing Page 6 ...
Processing Page 7 ...
Processing Page 8 ...
Pr

In [21]:
## You Use this if you want to run one at a time..

#i=2
#webscraper(mylinks[i],linkpages[i],'outfile_test.csv')

#webscraper(mylinks[0],4,'outfile_test.csv')

Processing Page 0 ...
NA Senior Data Engineer (Data Science and Artificial Intelligence)
Government support for those between jobs Applied Data Science Associate
Government support for those between jobs Data Science Lead, Large Customer Sales - Singapore
NA Data Science Engineer
NA Presales Specialist - Data Science
Government support for those between jobs Assistant Manager, Data Science
Government support for those between jobs Senior / Data Engineer (Data Science Team)
NA Data Science Consulting
Government support for those between jobs Data Science Engineer
No relevant experience required Data Science Intern
Government support for those between jobs Data Science Analyst
NA Senior Research Fellow (Institute of Data Science)
NA Senior Manager / Assistant Director (Data Science), NUHS Management Decision Support Office
NA Data Science Analyst
Government support for those between jobs Data Science Engineer
NA Research Associate (Data Science) - EEE / INFINITUS
Government support for t